# Training Grow-R Environment with DQN

## Importing Required Libraries

In [2]:
import os
import sys
import gymnasium as gym

# Add the parent directory to the path to allow for package imports
notebook_dir = os.getcwd()
sys.path.insert(0, os.path.abspath(os.path.join(notebook_dir, '..')))

from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common import logger
from training_utils import SaveOnIntervalCallback, visualise_training_logs
from plantos_env import PlantOSEnv

c:\Users\mohda\miniconda3\envs\rl_a3\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


## Define paths for saving models and logs

In [3]:
training_run = "1M"

# Define paths for saving models and logs
MODEL_DIR = os.path.join("DQN_Training/models", training_run)
LOG_DIR = os.path.join("DQN_Training/logs", training_run)
TENSORBOARD_LOG_DIR = "DQN_Training/logs"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

## Initialising the Environment

In [4]:
n_envs = 4
env_kwargs = {
    'grid_size': 21,
    'num_plants': 20,
    'num_obstacles': 12,
    'lidar_range': 6,
    'lidar_channels': 32,
    'observation_mode': 'grid',
    'thirsty_plant_prob': 0.5
}

# Create vectorized environment using the registered environment ID
env = make_vec_env('PlantOS-v0', n_envs=n_envs, env_kwargs=env_kwargs)

c:\Users\mohda\miniconda3\envs\rl_a3\Lib\site-packages\gymnasium\envs\registration.py:788: UserWarning: WARN: The environment is being initialised with render_mode='rgb_array' that is not in the possible render_modes ([]).
  logger.warn(


## Initialising the Model

In [5]:
# 2. Define the model
model = DQN(
    "MlpPolicy",
    env,
    learning_rate=5e-5,  # Slightly smaller learning rate for more stable updates
    buffer_size=100000,  # Increased buffer size from 100000
    learning_starts=10000, # Increase learning starts to fill the buffer a bit more
    batch_size=32,
    gamma=0.99,
    train_freq=4,
    gradient_steps=1,
    target_update_interval=1000,
    exploration_fraction=0.2,  # Explore for a bit longer
    exploration_final_eps=0.01, # Exploit more at the end
    verbose=1,
    policy_kwargs=dict(net_arch=[256, 256])
    #tensorboard_log=TENSORBOARD_LOG_DIR
)

# Log the training
new_logger = logger.configure(LOG_DIR, ["stdout", "csv"])
model.set_logger(new_logger)

Using cuda device
Logging to DQN_Training/logs\1M


## Setting up Callbacks

In [6]:
num_plants = env.get_attr('num_plants')[0]
r_goal = env.get_attr('R_GOAL')[0]
reward_threshold = r_goal * num_plants * 0.8
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=reward_threshold, verbose=1)

eval_callback = EvalCallback(
    env,
    callback_after_eval=stop_callback,
    best_model_save_path=MODEL_DIR,
    log_path=LOG_DIR,
    eval_freq=100000,
    deterministic=True,
    render=False
)

save_interval = 100000
save_callback = SaveOnIntervalCallback(save_interval, MODEL_DIR)
combined_callbacks = [eval_callback, save_callback]

c:\Users\mohda\miniconda3\envs\rl_a3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.num_plants to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_plants` for environment variables or `env.get_wrapper_attr('num_plants')` that will search the reminding wrappers.
  logger.warn(
c:\Users\mohda\miniconda3\envs\rl_a3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.R_GOAL to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.R_GOAL` for environment variables or `env.get_wrapper_attr('R_GOAL')` that will search the reminding wrappers.
  logger.warn(


## Training the Model

In [7]:
print("Starting DQN training with Stable Baselines3...")
model.learn(
    total_timesteps=200000, # 200,000
    callback=combined_callbacks
)
print("DQN Training Finished.")
print(f"Total timesteps trained: {model.num_timesteps}")

# Evaluate the trained model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(f"This is the avg reward: {mean_reward:.2f} +/- {std_reward:.2f}")

# 5. Save the final model
model.save(os.path.join(MODEL_DIR, f"dqn_plantos_final_model-{training_run}"))

env.close()

Starting DQN training with Stable Baselines3...
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 1e+03     |
|    ep_rew_mean      | -2.92e+03 |
|    exploration_rate | 0.901     |
| time/               |           |
|    episodes         | 4         |
|    fps              | 3493      |
|    time_elapsed     | 1         |
|    total_timesteps  | 4000      |
-----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 1e+03     |
|    ep_rew_mean      | -2.98e+03 |
|    exploration_rate | 0.802     |
| time/               |           |
|    episodes         | 8         |
|    fps              | 3519      |
|    time_elapsed     | 2         |
|    total_timesteps  | 8000      |
-----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 1e+03     |
|    ep_rew_mean      | -3.16e+03 |
|    exploration